In [1]:
!pip install torch
!pip install dgl==0.9.0
!pip install torch_geometric
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.3 MB/s eta 0:00:00


In [2]:
import os
import re
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from natsort import natsorted
import zipfile
import pandas as pd
import nltk
import string

In [3]:
# Download NLTK stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
# Define punctuation to remove
stopwords_list = stopwords.words('english')
additional_stopwords = ['33', '52']
stopwords_list.extend(additional_stopwords)

remove = string.punctuation + '’‘–—“”'

# Function to remove punctuation
def pun_remove(data):
    for char in remove:
        data = data.replace(char, ' ')
    return data

In [5]:
# Data Loader Class
class MySentences:
    def __init__(self, zip_file_path, extract_to):
        self.zip_file_path = zip_file_path
        self.extract_to = extract_to
        self.root_directory = self.unzip_and_get_root_directory()

    def unzip_and_get_root_directory(self):
        with zipfile.ZipFile(self.zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(self.extract_to)
        return os.path.join(self.extract_to, 'Image_data')

    def __iter__(self):
        file_list = [file for file in os.listdir(self.root_directory) if not file.startswith('.')]
        file_list = natsorted(file_list)
        for subdirectory in file_list:
            subdirectory_path = os.path.join(self.root_directory, subdirectory, 'txt')
            if os.path.exists(subdirectory_path):
                file_list_t = [file for file in os.listdir(subdirectory_path) if not file.startswith('.')]
                file_list_t = natsorted(file_list_t)
                for filename in file_list_t:
                    file_path = os.path.join(subdirectory_path, filename)
                    with open(file_path, 'r') as file:
                        A = []
                        for line in file:
                            line = pun_remove(line.lower())
                            line = word_tokenize(line)
                            line = [word for word in line if word not in stopwords_list]
                            if line:
                                A.append(line)
                        yield list(np.concatenate(A))


In [6]:
# Vocabulary and Tokenizer
class CustomTokenizer:
    def __init__(self, vocab):
        self.vocab = vocab
        self.pad_token = '[PAD]'
        self.unk_token = '[UNK]'
        self.cls_token = '[CLS]'
        self.mask_token = '[MASK]'

        # Add special tokens to vocab
        self.vocab[self.pad_token] = len(self.vocab)
        self.vocab[self.unk_token] = len(self.vocab)
        self.vocab[self.cls_token] = len(self.vocab)
        self.vocab[self.mask_token] = len(self.vocab)

        self.reverse_vocab = {v: k for k, v in self.vocab.items()}

    def encode(self, text):
        tokens = text.split()
        token_ids = [self.vocab.get(token, self.vocab[self.unk_token]) for token in tokens]
        return torch.tensor(token_ids, dtype=torch.long)

    def decode(self, token_ids):
        tokens = [self.reverse_vocab.get(id.item(), self.unk_token) for id in token_ids]
        return " ".join(tokens)


In [7]:
# Dataset
class TextDataset:
    def __init__(self, texts, tokenizer, max_len=50):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def process_all(self):
        token_ids_list = []
        for text in self.texts:
            token_ids = self.tokenizer.encode(text)
            token_ids = token_ids[:self.max_len]
            token_ids = torch.cat([
                token_ids,
                torch.tensor([self.tokenizer.vocab[self.tokenizer.pad_token]] * (self.max_len - len(token_ids)))
            ])
            token_ids_list.append(token_ids)

        # Stack all token IDs
        return torch.stack(token_ids_list)


In [8]:
# class BERTEmbeddingExtractor(nn.Module):
#     def __init__(self, vocab_size, embedding_dim, max_len):
#         super(BERTEmbeddingExtractor, self).__init__()
#         self.vocab_size = vocab_size
#         self.embedding_dim = embedding_dim
#         self.max_len = max_len

#         self.embeddings = nn.Embedding(vocab_size, embedding_dim)
#         self.transformer = nn.TransformerEncoder(
#             nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=4),
#             num_layers=1,
#         )
#         self.fc = nn.Linear(embedding_dim, vocab_size)

#     def forward(self, x):
#         x = self.embeddings(x)
#         x = self.transformer(x)
#         x = self.fc(x)
#         return x

#     def extract_embeddings(self, x):
#         """
#         Extract embeddings from the embedding layer.
#         """
#         return self.embeddings(x)


In [9]:
class BERTEmbeddingExtractor(nn.Module):
    def __init__(self, vocab_size, embedding_dim, max_len):
        super(BERTEmbeddingExtractor, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.max_len = max_len

        # Embedding layer
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)

        # Single Transformer Encoder Layer (not full encoder)
        self.transformer_layer = nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=2)

    def forward(self, x):
        x = self.embeddings(x)  # [Batch Size, Seq Len, Embedding Dim]
        x = self.transformer_layer(x)  # Pass through a single transformer layer
        return x

    def extract_embeddings(self, x):
        """
        Extract embeddings from the embedding layer.
        """
        return self.embeddings(x)

In [10]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [11]:
# Prepare data
zip_file_path = '/Image_data.zip'
extract_to = '/Extracted_image_data'
sentences = MySentences(zip_file_path, extract_to)
docs = [' '.join(sentence) for sentence in sentences]
print(f"Total number of documents: {len(docs)}")


Total number of documents: 106


In [12]:
# Build vocabulary
all_tokens = [token for doc in docs for token in word_tokenize(doc)]
word_counts = Counter(all_tokens)
vocab = {word: idx for idx, (word, _) in enumerate(word_counts.most_common())}

tokenizer = CustomTokenizer(vocab)
dataset = TextDataset(docs, tokenizer)

In [15]:
# Process all data
inputs = dataset.process_all()
inputs = inputs.long().to(device)

In [14]:
# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTEmbeddingExtractor(vocab_size=len(tokenizer.vocab), embedding_dim=100, max_len=50).to(device)


In [16]:
# Extract embeddings for the entire dataset
inputs = inputs.to(device)
embeddings = model.extract_embeddings(inputs)
embeddings = embeddings.detach().cpu().numpy()


In [17]:
# Mean pooling
sequence_embeddings = embeddings.mean(axis=1)


In [18]:
# Save embeddings to CSV
embeddings_df = pd.DataFrame(sequence_embeddings)
embeddings_df.to_csv("embeddings.csv", index=False)
print("Embeddings saved to embeddings.csv")

Embeddings saved to embeddings.csv


In [19]:
from google.colab import files

# Provide the path to the CSV file
file_path = "embeddings.csv"

# Trigger the download
files.download(file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import os
from natsort import natsorted
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
import string
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import numpy as np
from nltk.corpus import stopwords
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from gensim.models import FastText
import matplotlib.pyplot as plt
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
import numpy as np
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense,Bidirectional
from tensorflow.keras.regularizers import l2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
import numpy as np
from numpy.linalg import norm
from dgl.nn import GraphConv
from IPython.display import FileLink

import os
import zipfile
import numpy as np
from nltk.tokenize import word_tokenize
from natsort import natsorted
from nltk.corpus import stopwords
from transformers import BertTokenizer, BertModel
import torch
import pandas as pd

/usr/local/lib/python3.10/dist-packages/dgl/backend/pytorch/sparse.py:104: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=th.float16)
/usr/local/lib/python3.10/dist-packages/dgl/backend/pytorch/sparse.py:128: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, dZ):
/usr/local/lib/python3.10/dist-packages/dgl/backend/pytorch/sparse.py:177: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=th.float16)
/usr/local/lib/python3.10/dist-packages/dgl/backend/pytorch/sparse.py:207: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, *dZ):
/usr/local/lib/py

In [21]:
# Preparing Node Data
sequence_embeddings = list(sequence_embeddings)
Nodes_Data = pd.DataFrame({'Id': range(len(sequence_embeddings)), 'features': sequence_embeddings})
labels = pd.read_csv("/AnimalLabels.csv")
labels['majority_vote'] = labels.mode(axis=1, numeric_only=True).astype(int)
Nodes_Data['label'] = labels['majority_vote']

# Generating Edge Data for pairwise nodes with cosine similarity
Src_ID, Dst_ID, Src_feature, Dst_feature, edge_weight = [], [], [], [], []
for i in range(len(embeddings)):
    for j in range(i + 1, len(sequence_embeddings)):
        Src_ID.append(i)
        Dst_ID.append(j)
        src_feat, dst_feat = np.array(sequence_embeddings[i]), np.array(sequence_embeddings[j])
        similarity = np.dot(src_feat, dst_feat) / (norm(src_feat) * norm(dst_feat))
        Src_feature.append(src_feat)
        Dst_feature.append(dst_feat)
        edge_weight.append(max(similarity, 0))  # Ensure non-negative weights

Edge_Data = pd.DataFrame({
    'Src Id': Src_ID,
    'Dst Id': Dst_ID,
    'Src_feature': Src_feature,
    'Dst_feature': Dst_feature,
    'edge weights': edge_weight
})

# Save edge data to CSV
Edge_Data.to_csv('Edge_Data.csv', index=False)

# Provide download links
from IPython.display import FileLink
FileLink('Text_features_bert.csv'), FileLink('Edge_Data.csv')

(/content/Text_features_bert.csv, /content/Edge_Data.csv)

In [22]:
print(Edge_Data['edge weights'].describe())
ninetyfive_percentile = Edge_Data['edge weights'].quantile(0.95)
print("95th percentile of edge weights column:", ninetyfive_percentile)

count    5565.000000
mean        0.378166
std         0.402062
min         0.000000
25%         0.015399
50%         0.154173
75%         0.858200
max         1.000000
Name: edge weights, dtype: float64
95th percentile of edge weights column: 0.9874016404151916


In [23]:
percentiles=[round(0.000000,4),round(0.118725,4),round(0.826699,4),round(0.9842199087142944,4)]

In [24]:
import os
os.environ["DGLBACKEND"] = "pytorch"

import dgl
import torch
import numpy as np
from dgl.data import DGLDataset

# Assuming `Nodes_Data` and `Edge_Data` DataFrames are already defined

class CustomGraphDataset(DGLDataset):
    def __init__(self, threshold):
        self.threshold = threshold
        super().__init__(name="custom_graph")

    def process(self):
        # Define the list to store indices of edges to be removed
        edge_remove = []

        # Extract edge weights and apply threshold filtering
        edge_weights = Edge_Data["edge weights"].to_numpy()
        for i in range(len(edge_weights)):
            if edge_weights[i] <= self.threshold:
                edge_remove.append(i)
            else:
                edge_weight.append(edge_weights[i])

        # Convert node features and labels
        features_array = np.array(Nodes_Data["features"].tolist(), dtype=float)
        node_features = torch.from_numpy(features_array).float()
        node_labels = torch.tensor(Nodes_Data["label"].values, dtype=torch.long)

        # Convert edge features and source/destination IDs
        edge_features = torch.tensor(edge_weights, dtype=torch.float32)
        edges_src = torch.tensor(Edge_Data["Src Id"].values, dtype=torch.long)
        edges_dst = torch.tensor(Edge_Data["Dst Id"].values, dtype=torch.long)

        # Create the DGL graph
        self.graph = dgl.graph((edges_src, edges_dst), num_nodes=Nodes_Data.shape[0])
        self.graph.ndata["feat"] = node_features
        self.graph.ndata["label"] = node_labels
        self.graph.edata["weight"] = edge_features

        # Remove edges below threshold
        self.graph = dgl.remove_edges(self.graph, torch.tensor(edge_remove, dtype=torch.long))

        # Create train, validation, and test masks
        n_nodes = Nodes_Data.shape[0]
        n_train = int(n_nodes * 0.6)
        n_val = int(n_nodes * 0.2)
        train_mask = torch.zeros(n_nodes, dtype=torch.bool)
        val_mask = torch.zeros(n_nodes, dtype=torch.bool)
        test_mask = torch.zeros(n_nodes, dtype=torch.bool)
        train_mask[:n_train] = True
        val_mask[n_train:n_train + n_val] = True
        test_mask[n_train + n_val:] = True

        self.graph.ndata["train_mask"] = train_mask
        self.graph.ndata["val_mask"] = val_mask
        self.graph.ndata["test_mask"] = test_mask

    def __getitem__(self, idx):
        return self.graph

    def __len__(self):
        return 1

# Set threshold value
threshold = 0.9873
dataset = CustomGraphDataset(threshold)
graph = dataset[0]

# Print the graph to confirm its structure
print(graph)


Graph(num_nodes=106, num_edges=282,
      ndata_schemes={'feat': Scheme(shape=(100,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float32)})


In [25]:
# Verify column names in Edge_Data
print("Edge_Data columns:", Edge_Data.columns)

# Adjust column names here if necessary
src_col = 'Src Id'  # Replace with the actual name for source column if different
dst_col = 'Dst Id'  # Replace with the actual name for destination column if different
edge_weight_col = 'edge weights'  # Replace with the actual name for edge weight column if different

# Updated Code with Column Names Adjusted
new_g = dgl.compact_graphs(graph)
train_mask = graph.ndata["train_mask"]

# Set train_mask to True for the first 106 nodes
for i in range(106):
    train_mask[i] = True

# Set specified indices to False in train_mask
indices_to_change = [4, 105, 84, 27, 98, 88, 18, 65, 9, 2, 5, 49, 99, 69, 86, 67, 7, 28, 78, 70, 18, 74]
train_mask[indices_to_change] = False
graph.ndata["train_mask"] = train_mask

# Set test_mask as the inverse of train_mask
test_mask = ~train_mask
graph.ndata["test_mask"] = test_mask

# Filter Edge_Data for training edges (exclude edges connecting nodes in indices_to_change)
Edge_Data_train = Edge_Data[
    ~Edge_Data[src_col].isin(indices_to_change) &
    ~Edge_Data[dst_col].isin(indices_to_change) &
    (Edge_Data[edge_weight_col] > 0.8258)
]

# Filter Edge_Data for testing edges (include only edges connecting nodes in indices_to_change)
Edge_Data_test = Edge_Data[
    Edge_Data[src_col].isin(indices_to_change) &
    Edge_Data[dst_col].isin(indices_to_change) &
    (Edge_Data[edge_weight_col] > 0.8258)
]

# Create and save the dense adjacency matrix for the graph
adj_matrix = graph.adj(etype=None)
dense_matrix = adj_matrix.to_dense().numpy()

# Add node index as a new column to dense_matrix
new_column = np.arange(0, dense_matrix.shape[0]).reshape((106, 1))
dense_matrix = np.concatenate((new_column, dense_matrix), axis=1)

# Add row index as a new row to the top of dense_matrix
new_row = np.insert(np.arange(0, 106), 0, 0).reshape((1, 107))
result_matrix = np.vstack([new_row, dense_matrix])

# Save dense matrix to CSV
np.savetxt('dense_matrix_avg.csv', result_matrix, delimiter=',', fmt='%d')
FileLink(r'dense_matrix_avg.csv')

# Generate subgraph for training set and save its adjacency matrix
sg_train = dgl.node_subgraph(graph, train_mask)
sg_adjacency_train = sg_train.adj()
dense_matrix_sg_train = sg_adjacency_train.to_dense().numpy()

np.savetxt('dense_matrix_sg_train.csv', dense_matrix_sg_train, delimiter=',', fmt='%d')
FileLink(r'dense_matrix_sg_train.csv')

# Generate subgraph for test set and save its adjacency matrix
sg_test = dgl.node_subgraph(graph, test_mask)
sg_adjacency_test = sg_test.adj()
dense_matrix_sg_test = sg_adjacency_test.to_dense().numpy()

np.savetxt('dense_matrix_sg_test.csv', dense_matrix_sg_test, delimiter=',', fmt='%d')
FileLink(r'dense_matrix_sg_test.csv')


Edge_Data columns: Index(['Src Id', 'Dst Id', 'Src_feature', 'Dst_feature', 'edge weights'], dtype='object')


/content/dense_matrix_sg_test.csv

In [26]:
seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats, norm='both')
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        in_feat = in_feat.float()
        h = self.conv1(g, in_feat)
        h = F.leaky_relu(h)
        h = self.conv2(g, h)
        return h

In [27]:
los=[]
train_accuracy=[]
test_accuracy=[]
epoch=[]

sg_train = dgl.add_self_loop(sg_train)
sg_test = dgl.add_self_loop(sg_test)

def train(sg_train, sg_test, model):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=1)
    best_train_acc = 0
    best_test_acc = 0

    features_train = sg_train.ndata["feat"]
    labels = graph.ndata["label"]
    train_mask = graph.ndata["train_mask"]
    test_mask = graph.ndata["test_mask"]

    for e in range(1000):
        # Forward pass
        logits = model(sg_train, features_train)
        pred = logits.argmax(1)

        labels = graph.ndata["label"].long()
        loss = F.cross_entropy(logits, labels[train_mask])

        # Test set predictions
        with torch.no_grad():
            model.eval()
            logits_test = model(sg_test, sg_test.ndata["feat"])
            pred_test = logits_test.argmax(1)

        # Calculate accuracies
        train_acc = (pred == labels[train_mask]).float().mean()
        test_acc = (pred_test == labels[test_mask]).float().mean()

        # Track best train and test accuracy
        best_train_acc = max(best_train_acc, train_acc)
        best_test_acc = max(best_test_acc, test_acc)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Log metrics every 5 epochs
        if e % 5 == 0:
            epoch.append(e)
            los.append(round(loss.item(), 3))
            train_accuracy.append(round(train_acc.item(), 3))
            test_accuracy.append(round(test_acc.item(), 3))
            print(
                f"In epoch {e}, loss: {loss:.3f}, train acc: {train_acc:.3f} (train {best_train_acc:.3f}), test acc: {test_acc:.3f} (best {best_test_acc:.3f})"
            )

# Initialize and train the model
model = GCN(graph.ndata["feat"].shape[1], 22, 2)
train(sg_train, sg_test, model)

/usr/local/lib/python3.10/dist-packages/dgl/backend/pytorch/tensor.py:352: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), "Cannot convert view " \


In epoch 0, loss: 0.806, train acc: 0.435 (train 0.435), test acc: 0.286 (best 0.286)
In epoch 5, loss: 7.224, train acc: 0.588 (train 0.647), test acc: 0.571 (best 0.667)
In epoch 10, loss: 1.930, train acc: 0.506 (train 0.647), test acc: 0.476 (best 0.667)
In epoch 15, loss: 2.496, train acc: 0.659 (train 0.682), test acc: 0.714 (best 0.714)
In epoch 20, loss: 0.612, train acc: 0.659 (train 0.682), test acc: 0.667 (best 0.714)
In epoch 25, loss: 0.557, train acc: 0.671 (train 0.682), test acc: 0.619 (best 0.714)
In epoch 30, loss: 0.454, train acc: 0.741 (train 0.741), test acc: 0.571 (best 0.714)
In epoch 35, loss: 0.446, train acc: 0.753 (train 0.753), test acc: 0.619 (best 0.714)
In epoch 40, loss: 0.380, train acc: 0.765 (train 0.788), test acc: 0.619 (best 0.714)
In epoch 45, loss: 4.481, train acc: 0.741 (train 0.847), test acc: 0.429 (best 0.714)
In epoch 50, loss: 0.859, train acc: 0.647 (train 0.847), test acc: 0.333 (best 0.714)
In epoch 55, loss: 0.400, train acc: 0.741 (t

In [28]:
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import torch
from dgl.data import DGLDataset


class KarateClubDataset(DGLDataset):
    def __init__(self,params):
        self.params=params
        #self.threshold=threshold
        super().__init__(name="karate_club")
        self.process()
    def process(self):
        edge_remove=[]
        C=edge_weights
        for i in range(0,len(C)):
            if C[i]<=self.params['percentile']:
                edge_remove.append(i)
        nodes_data = Nodes_Data
        edges_data = Edge_Data
        features_array = np.array(nodes_data["features"].tolist(), dtype=float)
        node_features = torch.from_numpy(features_array)
        node_labels = torch.from_numpy(
                      nodes_data["label"].astype("category").cat.codes.to_numpy()
                       ).clone().detach()
        edge_features = torch.from_numpy(edges_data["edge weights"].to_numpy())
        edges_src = torch.from_numpy(edges_data["Src Ids"].to_numpy())
        edges_dst = torch.from_numpy(edges_data["Dst Ids"].to_numpy())
        self.graph = dgl.graph(
            (edges_src, edges_dst), num_nodes=nodes_data.shape[0]
        )
        self.graph.ndata["feat"] = node_features
        self.graph.ndata["label"] = node_labels
        self.graph.edata["weight"] = edge_features

        self.graph=dgl.remove_edges(self.graph, torch.tensor(edge_remove))
        n_nodes = nodes_data.shape[0]
        n_train = int(n_nodes * 0.6)
        n_val = int(n_nodes * 0.2)
        train_mask = torch.zeros(n_nodes, dtype=torch.bool)
        val_mask = torch.zeros(n_nodes, dtype=torch.bool)
        test_mask = torch.zeros(n_nodes, dtype=torch.bool)
        train_mask[:n_train] = True
        val_mask[n_train : n_train + n_val] = True
        test_mask[n_train + n_val :] = True
        self.graph.ndata["train_mask"] = train_mask
        self.graph.ndata["val_mask"] = val_mask
        self.graph.ndata["test_mask"] = test_mask

    def __getitem__(self, i):
        return self.graph

    def __len__(self):
        return 1

In [29]:
import os
import dgl
import torch
import numpy as np
from dgl.data import DGLDataset

os.environ["DGLBACKEND"] = "pytorch"

class KarateClubDataset(DGLDataset):
    def __init__(self, params):
        self.params = params
        super().__init__(name="karate_club")
        self.process()

    def process(self):
        # Edge removal based on the percentile value
        edge_remove = []
        C = edge_weights  # Assuming `edge_weights` is predefined elsewhere
        for i in range(len(C)):
            if C[i] <= self.params['percentile']:
                edge_remove.append(i)

        nodes_data = Nodes_Data  # Assuming `Nodes_Data` is a predefined dataframe
        edges_data = Edge_Data  # Assuming `Edge_Data` is a predefined dataframe

        # Prepare node features, labels, and edge features
        features_array = np.array(nodes_data["features"].tolist(), dtype=float)
        node_features = torch.from_numpy(features_array)
        node_labels = torch.from_numpy(
            nodes_data["label"].astype("category").cat.codes.to_numpy()
        ).clone().detach()

        edge_features = torch.from_numpy(edges_data["edge weights"].to_numpy())
        edges_src = torch.from_numpy(edges_data["Src Ids"].to_numpy())
        edges_dst = torch.from_numpy(edges_data["Dst Ids"].to_numpy())

        # Create the graph
        self.graph = dgl.graph(
            (edges_src, edges_dst), num_nodes=nodes_data.shape[0]
        )
        self.graph.ndata["feat"] = node_features
        self.graph.ndata["label"] = node_labels
        self.graph.edata["weight"] = edge_features

        # Remove edges based on edge removal condition
        self.graph = dgl.remove_edges(self.graph, torch.tensor(edge_remove))

        # Add self-loops to prevent 0-in-degree error
        self.graph = dgl.add_self_loop(self.graph)

        # Splitting into train, validation, and test masks
        n_nodes = nodes_data.shape[0]
        n_train = int(n_nodes * 0.6)
        n_val = int(n_nodes * 0.2)

        train_mask = torch.zeros(n_nodes, dtype=torch.bool)
        val_mask = torch.zeros(n_nodes, dtype=torch.bool)
        test_mask = torch.zeros(n_nodes, dtype=torch.bool)

        train_mask[:n_train] = True
        val_mask[n_train:n_train + n_val] = True
        test_mask[n_train + n_val:] = True

        self.graph.ndata["train_mask"] = train_mask
        self.graph.ndata["val_mask"] = val_mask
        self.graph.ndata["test_mask"] = test_mask

    def __getitem__(self, i):
        return self.graph

    def __len__(self):
        return 1


In [30]:
class GCN(nn.Module):
    def __init__(self, in_feats, params, num_classes):
        super(GCN, self).__init__()
        # Initialize the layers based on provided params
        self.conv1 = GraphConv(in_feats, params['output_features'])
        self.conv2 = GraphConv(params['output_features'], num_classes)

    def forward(self, g, in_feat, params):
        # Ensure features are in float format
        in_feat = in_feat.float()

        # Perform first graph convolution
        h = self.conv1(g, in_feat)

        # Apply activation function based on `params`
        h = getattr(F, params['activation'])(h)  # Dynamic activation based on params

        # Perform second graph convolution
        h = self.conv2(g, h)
        return h

In [31]:
import torch.optim as optim
import torch.nn.functional as F

# Lists to store metrics for later use
los = []
train_accuracy = []
test_accuracy = []
TT = []
CT = []
epoch = []

# Set random seeds for reproducibility
seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

def train(g, sg_train, sg_test, model, params):
    model.train()

    # Optimizer setup
    optimizer = getattr(optim, params['optimizer'])(model.parameters(), lr=params['learning_rate'])

    # Initialize tracking variables for best accuracy
    best_train_acc = 0
    best_test_acc = 0
    corresponding_test = 0

    # Retrieve the relevant graph and node data
    features_train = sg_train.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    test_mask = g.ndata["test_mask"]

    # Training loop
    for e in range(100):
        # Forward pass
        logits = model(sg_train, features_train, params)

        # Compute prediction
        pred = logits.argmax(1)

        # Loss calculation
        if params['loss'] == 'binary_cross_entropy':
            loss = getattr(F, params['loss'])(torch.sigmoid(logits), labels[train_mask])
        else:
            loss = getattr(F, params['loss'])(logits, labels[train_mask])

        # Evaluate performance on the test graph
        with torch.no_grad():
            model.eval()
            logits_test = model(sg_test, sg_test.ndata["feat"], params)

        pred_test = logits_test.argmax(1)

        # Calculate training and testing accuracies
        train_acc = (pred == labels[train_mask]).float().mean()
        test_acc = (pred_test == labels[test_mask]).float().mean()

        # Track the best accuracies
        if best_train_acc < train_acc:
            best_train_acc = train_acc
            corresponding_test = test_acc
        if best_test_acc < test_acc:
            best_test_acc = test_acc

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Logging every 5 epochs
        if e % 5 == 0:
            epoch.append(e)
            los.append(round(loss.item(), 3))
            train_accuracy.append(round(train_acc.item(), 3))

    # Store results for later
    TT.append({'train_accuracy': best_train_acc})
    CT.append({'test_accuracy': corresponding_test})

    # Print final accuracy results
    print(f'Train Accuracy: {best_train_acc}, Test Accuracy: {corresponding_test}')

    return best_train_acc

In [32]:
!pip install optuna
import optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.6 MB/s eta 0:00:00


In [33]:
# import torch
# import optuna
# import dgl
# from dgl.data import KarateClubDataset

def objective(trial):
    # Define the possible values for each hyperparameter
    percentile_values = [percentiles[0], percentiles[1], percentiles[2], percentiles[3]]
    learning_rate = [0.0001,0.001,0.01,0.1]
    output_features = [10, 12, 14, 16, 18, 20, 22]

    params = {
    'output_features': trial.suggest_categorical('output_features', output_features),
    'optimizer': trial.suggest_categorical('optimizer', ["Adam", "RMSprop", "SGD"]),
    'percentile': trial.suggest_categorical('percentile', percentile_values),
    'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True),
    'loss': trial.suggest_categorical('loss', ['cross_entropy']),
    'activation': trial.suggest_categorical('activation', ['relu', 'selu', 'elu']),
    'dropout': trial.suggest_float('dropout', 0.2, 0.5),  # Adding dropout hyperparameter
}

    # Load dataset and setup graph
    dataset = KarateClubDataset()
    g = dataset[0]

    # Initialize node features as random values (if not already present in the dataset)
    if "feat" not in g.ndata:
        g.ndata["feat"] = torch.randn(g.number_of_nodes(), 10)  # Here, we use 10 as the feature dimension, but adjust as necessary

    # Add self-loop to avoid zero-in-degree nodes
    g = dgl.add_self_loop(g)

    # Initialize train_mask and test_mask
    g.ndata["train_mask"] = torch.zeros(g.number_of_nodes(), dtype=torch.bool)
    g.ndata["test_mask"] = torch.zeros(g.number_of_nodes(), dtype=torch.bool)

    # Modify train_mask for training set (set to True for the available nodes, up to g.number_of_nodes())
    train_mask = g.ndata["train_mask"]
    for i in range(g.number_of_nodes()):  # Use number of nodes instead of fixed 106
        train_mask[i] = True

    # Manually set indices to False for testing set (ensure indices are within bounds)
    indices_to_change = [4, 105, 84, 27, 98, 88, 18, 65, 9, 2, 5, 49, 99, 69, 86, 67, 7, 28, 78, 70, 18, 74]
    # Filter out indices that exceed the available number of nodes
    indices_to_change = [i for i in indices_to_change if i < g.number_of_nodes()]
    train_mask[indices_to_change] = False
    g.ndata["train_mask"] = train_mask
    test_mask = ~train_mask
    g.ndata["test_mask"] = test_mask

    # Create subgraphs for training and testing
    sg_train = dgl.node_subgraph(g, train_mask)
    sg_test = dgl.node_subgraph(g, test_mask)

    # Initialize GCN model with parameters from the trial
    model = GCN(g.ndata["feat"].shape[1], params, 2)

    # Train the model and get the best accuracy
    accuracy = train(g, sg_train, sg_test, model, params)

    return accuracy


In [34]:
# def objective(trial):
#     percentile_values=[percentiles[0],percentiles[1],percentiles[2],percentiles[3]]
#     learning_rate=[0.01,0.1,1,10,100,200]
#     output_features=[10,12,14,16,18,20,22]
#     params={'output_features':trial.suggest_categorical('output_features',output_features),
#            'optimizer':trial.suggest_categorical('optimizer',["Adam", "RMSprop", "SGD"]),
#            'percentile':trial.suggest_categorical('percentile',percentile_values),
#            'learning_rate':trial.suggest_categorical('learning_rate',learning_rate),
#            'loss':trial.suggest_categorical('loss',['cross_entropy']),
#            'activation':trial.suggest_categorical('activation',['relu','selu','elu','leaky_relu','tanh'])}
#     dataset = KarateClubDataset(params)
#     g = dataset[0]
#     train_mask = g.ndata["train_mask"]
#     for i in range(106):
#         train_mask[i] = True

#     indices_to_change = [4, 105, 84, 27, 98, 88, 18, 65, 9, 2, 5, 49, 99, 69, 86, 67, 7, 28, 78, 70, 18, 74]
#     train_mask[indices_to_change] = False
#     g.ndata["train_mask"]=train_mask
#     test_mask = ~train_mask
#     g.ndata["test_mask"]=test_mask
#     sg_train=dgl.node_subgraph(g, train_mask)
#     sg_test = dgl.node_subgraph(g, test_mask)
#     model=GCN(g.ndata["feat"].shape[1],params, 2)
#     accuracy=train(g,sg_train,sg_test,model,params)
#     return accuracy

import torch
import optuna
import dgl
from dgl.data import KarateClubDataset

def objective(trial):
    # Define the possible values for each hyperparameter
    percentile_values = [percentiles[0], percentiles[1], percentiles[2], percentiles[3]]
    learning_rate = [0.0001,0.001,0.01,0.1]
    output_features = [10, 12, 14, 16, 18, 20, 22]

    # # Define the params dictionary with values for hyperparameter optimization
    # params = {
    #     'output_features': trial.suggest_categorical('output_features', output_features),
    #     'optimizer': trial.suggest_categorical('optimizer', ["Adam", "RMSprop", "SGD"]),
    #     'percentile': trial.suggest_categorical('percentile', percentile_values),
    #     'learning_rate': trial.suggest_categorical('learning_rate', learning_rate),
    #     'loss': trial.suggest_categorical('loss', ['cross_entropy']),
    #     'activation': trial.suggest_categorical('activation', ['relu', 'selu', 'elu', 'leaky_relu', 'tanh'])
    # }

    params = {
    'output_features': trial.suggest_categorical('output_features', output_features),
    'optimizer': trial.suggest_categorical('optimizer', ["Adam", "RMSprop", "SGD"]),
    'percentile': trial.suggest_categorical('percentile', percentile_values),
    'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True),
    'loss': trial.suggest_categorical('loss', ['cross_entropy']),
    'activation': trial.suggest_categorical('activation', ['relu', 'selu', 'elu']),
    'dropout': trial.suggest_float('dropout', 0.2, 0.5),  # Adding dropout hyperparameter
}

    # Load dataset and setup graph
    dataset = KarateClubDataset()
    g = dataset[0]

    # Initialize node features as random values (if not already present in the dataset)
    if "feat" not in g.ndata:
        g.ndata["feat"] = torch.randn(g.number_of_nodes(), 10)  # Here, we use 10 as the feature dimension, but adjust as necessary

    # Add self-loop to avoid zero-in-degree nodes
    g = dgl.add_self_loop(g)

    # Initialize train_mask and test_mask
    g.ndata["train_mask"] = torch.zeros(g.number_of_nodes(), dtype=torch.bool)
    g.ndata["test_mask"] = torch.zeros(g.number_of_nodes(), dtype=torch.bool)

    # Modify train_mask for training set (set to True for the available nodes, up to g.number_of_nodes())
    train_mask = g.ndata["train_mask"]
    for i in range(g.number_of_nodes()):  # Use number of nodes instead of fixed 106
        train_mask[i] = True

    # Manually set indices to False for testing set (ensure indices are within bounds)
    indices_to_change = [4, 105, 84, 27, 98, 88, 18, 65, 9, 2, 5, 49, 99, 69, 86, 67, 7, 28, 78, 70, 18, 74]
    # Filter out indices that exceed the available number of nodes
    indices_to_change = [i for i in indices_to_change if i < g.number_of_nodes()]
    train_mask[indices_to_change] = False
    g.ndata["train_mask"] = train_mask
    test_mask = ~train_mask
    g.ndata["test_mask"] = test_mask

    # Create subgraphs for training and testing
    sg_train = dgl.node_subgraph(g, train_mask)
    sg_test = dgl.node_subgraph(g, test_mask)

    # Initialize GCN model with parameters from the trial
    model = GCN(g.ndata["feat"].shape[1], params, 2)

    # Train the model and get the best accuracy
    accuracy = train(g, sg_train, sg_test, model, params)

    return accuracy


In [35]:
# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=2520)
import time

# Initialize the study
study = optuna.create_study(direction="maximize")

# Measure the total time for all trials
start_time = time.time()  # Start time before optimization

# Perform the optimization
study.optimize(objective, n_trials=2520)

end_time = time.time()  # End time after optimization

# Calculate total compute time
total_compute_time = end_time - start_time

print(f"Total computation time for all 2520 trials: {total_compute_time:.4f} seconds")

[I 2024-12-09 00:36:32,276] A new study created in memory with name: no-name-910bb85e-af8a-4a23-b42f-63cb0d73e921
[I 2024-12-09 00:36:33,431] Trial 0 finished with value: 0.9615384340286255 and parameters: {'output_features': 20, 'optimizer': 'Adam', 'percentile': 0.1187, 'learning_rate': 0.00038926228961729605, 'loss': 'cross_entropy', 'activation': 'elu', 'dropout': 0.40854693793697117}. Best is trial 0 with value: 0.9615384340286255.


Train Accuracy: 0.9615384340286255, Test Accuracy: 0.75


[I 2024-12-09 00:36:34,522] Trial 1 finished with value: 0.5384615659713745 and parameters: {'output_features': 10, 'optimizer': 'SGD', 'percentile': 0.1187, 'learning_rate': 3.0066412107036897e-05, 'loss': 'cross_entropy', 'activation': 'selu', 'dropout': 0.3348793056964121}. Best is trial 0 with value: 0.9615384340286255.


Train Accuracy: 0.5384615659713745, Test Accuracy: 0.75


[I 2024-12-09 00:36:35,956] Trial 2 finished with value: 1.0 and parameters: {'output_features': 16, 'optimizer': 'RMSprop', 'percentile': 0.9842, 'learning_rate': 0.003715617685947701, 'loss': 'cross_entropy', 'activation': 'elu', 'dropout': 0.28352612525596244}. Best is trial 2 with value: 1.0.


...

Train Accuracy: 1.0, Test Accuracy: 0.5


[I 2024-12-09 01:12:21,711] Trial 2518 finished with value: 1.0 and parameters: {'output_features': 16, 'optimizer': 'Adam', 'percentile': 0.9842, 'learning_rate': 0.04340212145007476, 'loss': 'cross_entropy', 'activation': 'selu', 'dropout': 0.23326079896860233}. Best is trial 2 with value: 1.0.


Train Accuracy: 1.0, Test Accuracy: 0.5


[I 2024-12-09 01:12:22,914] Trial 2519 finished with value: 1.0 and parameters: {'output_features': 20, 'optimizer': 'Adam', 'percentile': 0.1187, 'learning_rate': 0.018279516598810403, 'loss': 'cross_entropy', 'activation': 'elu', 'dropout': 0.3133982706682213}. Best is trial 2 with value: 1.0.


Train Accuracy: 1.0, Test Accuracy: 0.25
Total computation time for all 2520 trials: 2150.6365 seconds


In [36]:
import heapq

# Assuming CT is a list of dictionaries and each dictionary has a 'test_accuracy' key
top_ten_max_results = heapq.nlargest(10, enumerate(CT), key=lambda x: x[1]['test_accuracy'])

# Print the top ten results along with their indices
for index, result in top_ten_max_results:
    print(f'Index: {index}, Test Accuracy: {result["test_accuracy"]}')

Index: 686, Test Accuracy: 1.0
Index: 2243, Test Accuracy: 1.0
Index: 118, Test Accuracy: 0.875
Index: 119, Test Accuracy: 0.875
Index: 398, Test Accuracy: 0.875
Index: 424, Test Accuracy: 0.875
Index: 513, Test Accuracy: 0.875
Index: 531, Test Accuracy: 0.875
Index: 611, Test Accuracy: 0.875
Index: 629, Test Accuracy: 0.875
